In [2]:
import paho.mqtt.client as mqtt
import time
from datetime import datetime
import os

mqtt_broker_host = "150.214.203.56"
mqtt_broker_port = 1883

# Lista de archivos y sus respectivos tipos de sensor
sensor_files = {
    "closet_2.tsv": "Closet_Room_1",
    "closet_3.tsv": "Closet_Room_2",
    "closet_4.tsv": "Closet_Room_3",
    "dishes_9.tsv": "Dishwasher",
    "fridge_13.tsv": "Fridge",
    "micro_5.tsv": "Microwave",
    "pans_8.tsv": "Pans",
    "sink_21.tsv": "Sink",
    "tap_22.tsv": "Tap",
    "wc_17.tsv": "WC"
}

def on_publish(client, userdata, mid):
    print(f"Mensaje publicado con éxito (ID: {mid})")

# Configuración del cliente MQTT
client = mqtt.Client()
client.on_publish = on_publish

# Conexión al broker MQTT
client.connect(mqtt_broker_host, mqtt_broker_port, keepalive=60)
client.loop_start()

def read_sensor_data(file_path, sensor_type):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        sensor_data_list = []
        for line in lines:
            line = line.strip()
            if not line:  # Ignorar líneas vacías
                continue
            try:
                timestamp, state = line.split('\t')
                # Convertir el timestamp a formato ISO 8601
                timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S").isoformat()
                sensor_data_list.append((f"{timestamp}\t{state} {sensor_type}", float(state)))
            except ValueError:
                print(f"Línea incorrecta en el archivo {file_path}: {line}")
                continue
        # Ordenar la lista de datos del sensor, poniendo las líneas con 1.0 al principio
        sensor_data_list.sort(key=lambda x: -x[1])
        return [data[0] for data in sensor_data_list]

try:
    device_id_counters = {sensor_type: 1 for sensor_type in sensor_files.values()}  # Inicializa el contador para cada tipo de sensor
    processed_data = {file_name: set() for file_name in sensor_files}  # Para almacenar las líneas ya procesadas

    while True:
        all_processed = True  # Variable para verificar si todos los archivos han sido completamente procesados

        for file_name, sensor_type in sensor_files.items():
            if os.path.exists(file_name):
                sensor_data_list = read_sensor_data(file_name, sensor_type)
                for sensor_data in sensor_data_list:
                    if sensor_data in processed_data[file_name]:
                        continue  # Saltar si ya se ha procesado

                    all_processed = False  # Hay datos que aún necesitan ser procesados

                    try:
                        timestamp, state_type = sensor_data.split('\t')
                        state, sensor_type = state_type.split()
                    except ValueError:
                        print(f"Error procesando los datos del sensor: {sensor_data}")
                        continue

                    # Crea un device_id único manteniendo el nombre del tipo de sensor y añadiendo un identificador único
                    device_id = f"binarySensor{sensor_type}{device_id_counters[sensor_type]:03d}"
                    topic = f"/ul/4jggokgpepnvsb2uv4s40d59ov/{device_id}/attrs"

                    result = client.publish(topic, f"timestamp_sensor|{timestamp}|activo|{state}|tipo_sensor|{sensor_type}", qos=2)
                    print("mqtt_topic->", topic, "sensor_data->", sensor_data, "result->", result)

                    # Marcar la línea como procesada
                    processed_data[file_name].add(sensor_data)
                    # Incrementa el contador para el tipo de sensor actual
                    device_id_counters[sensor_type] += 1

                    time.sleep(1)  # Espera antes de la siguiente lectura

        if all_processed:
            print("Todos los datos han sido procesados. Terminando ejecución.")
            break

        time.sleep(5)  # Espera antes de la siguiente iteración

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
    client.disconnect()

mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1001/attrs sensor_data-> 2024-05-10T20:00:00	1.0 Closet_Room_1 result-> (0, 1)
Mensaje publicado con éxito (ID: 1)
mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1002/attrs sensor_data-> 2024-05-10T02:00:00	0.0 Closet_Room_1 result-> (0, 2)
Mensaje publicado con éxito (ID: 2)
mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1003/attrs sensor_data-> 2024-05-10T02:01:00	0.0 Closet_Room_1 result-> (0, 3)
Mensaje publicado con éxito (ID: 3)
mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1004/attrs sensor_data-> 2024-05-10T02:02:00	0.0 Closet_Room_1 result-> (0, 4)
Mensaje publicado con éxito (ID: 4)
mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1005/attrs sensor_data-> 2024-05-10T02:03:00	0.0 Closet_Room_1 result-> (0, 5)
Mensaje publicado con éxito (ID: 5)
mqtt_topic-> /ul/4jggokgpepnvsb2uv4s40d59ov/binarySensorCloset_Room_1006/attrs sensor

In [ ]:
%reset -f